In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import files
import time
import concurrent.futures
from datetime import datetime
import os
import psutil

# Fetch the webpage
url_template = "https://pgmall.my/category?path=1&page={}"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

csv_file = "Item_list.csv"
performance_log_file = "performance_log.csv"
version_label = "Multithreading"

# Performance tracking setup
process = psutil.Process(os.getpid())
start_cpu = psutil.cpu_percent(interval=1)
start_mem = process.memory_info().rss / (1024 * 1024)  # MB

def scrape_page(page_number):
    """Scrape a single page with retry logic"""
    url = url_template.format(page_number)
    retries = 3

    for attempt in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 5))
            print(f"Rate limited on page {page_number}, retrying in {retry_after} seconds... (Attempt {attempt+1}/{retries})")
            time.sleep(retry_after)
            continue
        elif response.status_code != 200:
            print(f"Failed to fetch page {page_number}. Status code: {response.status_code}")
            return []

        # Parse and extract data
        soup = BeautifulSoup(response.content, 'html.parser')
        listings = soup.find_all('div', class_='category_product_col_new p-div')

        data = []
        for listing in listings:
            # Extract the product link
            link_tag = listing.find('a', href=True)
            link = link_tag['href'] if link_tag else None

            # Extract the product name
            name_tag = listing.find('p', class_='p-name text-left text-darkgrey')
            product_name = name_tag.text.strip() if name_tag else None

            # Extract price
            price_tag = listing.find('span', class_='p-price-red p-overflow')
            price = price_tag.text.strip() if price_tag else None

            # Extract location
            location_tag = listing.find('div',class_="text-left color-grey")
            location = location_tag.text.strip() if location_tag else None

            # Append the extracted data to the list
            data.append({
                "product_name": product_name,
                "link": link,
                "price": price,
                "location": location,
            })

        return data

def save_to_csv(data, csv_file="Item_list.csv"):
    """Save data to CSV"""
    # Check if the file exists and has headers
    file_exists = os.path.isfile(csv_file)

    with open(csv_file, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys())
        if not file_exists:
            writer.writeheader()
        writer.writerows(data)

def log_performance(version, total_data_saved, elapsed_time, start_cpu, end_cpu, start_mem, end_mem):
    throughput = total_data_saved / elapsed_time if elapsed_time > 0 else 0
    with open(performance_log_file, "a", newline="") as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(["version", "total_records", "total_time", "cpu_start", "cpu_end", "mem_start", "mem_end", "throughput"])
        writer.writerow([
            version,
            total_data_saved,
            elapsed_time,
            start_cpu,
            end_cpu,
            start_mem,
            end_mem,
            throughput
        ])


# Main function with rate-limiting and data limit
def main():
    page_number = 1
    max_workers = 5  # Reduced concurrency to prevent rate limits
    request_delay = 0  # Seconds between requests
    max_data_limit = 200000  # Maximum number of data points to extract
    total_data_saved = 0  # Counter for successfully saved data points
    start_time = time.time()

    csv_file = "Item_list.csv"

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        while True:
            print(f"Scraping page {page_number}...")
            future = executor.submit(scrape_page, page_number)

            try:
                property_data = future.result()
                if not property_data:
                    print(f"No data found on page {page_number}. Exiting...")
                    break

                # Save scraped data to CSV
                save_to_csv(property_data)
                total_data_saved += len(property_data)
                print(f"Page {page_number} scraped and saved ({len(property_data)} items). Total: {total_data_saved}")
                # Check if the data limit has been reached
                if total_data_saved >= max_data_limit:
                    print(f"Reached the maximum data limit of {max_data_limit}. Exiting...")
                    break

                page_number += 1
                time.sleep(request_delay)
            except Exception as e:
                print(f"Error scraping page {page_number}: {e}")
                break

    end_time = time.time()
    elapsed_time = end_time - start_time
    end_cpu = psutil.cpu_percent(interval=1)
    end_mem = process.memory_info().rss / (1024 * 1024)

    print(f"\nTotal records scraped: {total_data_saved}")
    print(f"Total time taken: {elapsed_time:.2f} seconds")
    print(f"Start memory: {start_mem:.2f} MB, End memory: {end_mem:.2f} MB")
    print(f"Start CPU: {start_cpu}%, End CPU: {end_cpu}%")
    print(f"Records per second: {total_data_saved / elapsed_time:.2f} rec/sec")

    log_performance(version_label, total_data_saved, elapsed_time, start_cpu, end_cpu, start_mem, end_mem)

    # Final cleanup and download (only in Google Colab)
    if os.path.isfile(csv_file):
        if 'google.colab' in str(get_ipython()):
            files.download(csv_file)
            if os.path.isfile(performance_log_file):
              files.download(performance_log_file)
        else:
            print(f"CSV file saved locally: {csv_file}")
    else:
        print("No data was scraped. CSV file was not created.")

if __name__ == "__main__":
        main()

Scraping page 1...
Page 1 scraped and saved (50 items). Total: 50
Scraping page 2...
Page 2 scraped and saved (50 items). Total: 100
Scraping page 3...
Page 3 scraped and saved (50 items). Total: 150
Scraping page 4...
Page 4 scraped and saved (50 items). Total: 200
Scraping page 5...
Page 5 scraped and saved (50 items). Total: 250
Scraping page 6...
Page 6 scraped and saved (50 items). Total: 300
Scraping page 7...
Page 7 scraped and saved (50 items). Total: 350
Scraping page 8...
Page 8 scraped and saved (50 items). Total: 400
Scraping page 9...
Page 9 scraped and saved (50 items). Total: 450
Scraping page 10...
Page 10 scraped and saved (50 items). Total: 500
Scraping page 11...
Page 11 scraped and saved (50 items). Total: 550
Scraping page 12...
Page 12 scraped and saved (50 items). Total: 600
Scraping page 13...
Page 13 scraped and saved (50 items). Total: 650
Scraping page 14...
Page 14 scraped and saved (50 items). Total: 700
Scraping page 15...
Page 15 scraped and saved (50 ite